In [12]:
import time

# Define the interface for the Real Subject
class DatabaseQuery:
    def execute_query(self, query):
        pass

# Real Subject: Represents the actual database
class RealDatabaseQuery(DatabaseQuery):
    def execute_query(self, query):
        print(f"Executing query: {query}")
        # Simulate a database query and return the results
        return f"Results for query: {query}"

# Proxy: Caching Proxy for Database Queries
class CacheProxy(DatabaseQuery):
    def __init__(self, real_database_query, cache_duration_seconds):
        self._real_database_query = real_database_query
        self._cache = {}
        self._cache_duration = cache_duration_seconds

    def execute_query(self, query):
        if (
            query in self._cache
            and time.time() - self._cache[query]["timestamp"] <= self._cache_duration
        ):
            # Return cached result if it's still valid
            print(f"CacheProxy: Returning cached result for query: {query}")
            return self._cache[query]["result"]
        else:
            # Execute the query and cache the result
            result = self._real_database_query.execute_query(query)
            self._cache[query] = {"result": result, "timestamp": time.time()}
            return result

# Client code
if __name__ == "__main__":
    # Create the Real Subject
    real_database_query = RealDatabaseQuery()

    # Create the Cache Proxy with a cache duration of 5 seconds
    cache_proxy = CacheProxy(real_database_query, cache_duration_seconds=5)

    # Perform database queries, some of which will be cached
    print(cache_proxy.execute_query("SELECT * FROM table1"))
    print(cache_proxy.execute_query("SELECT * FROM table2"))
    time.sleep(3)  # Sleep for 3 seconds

    # Should return cached result
    print(cache_proxy.execute_query("SELECT * FROM table1"))

    print(cache_proxy.execute_query("SELECT * FROM table3"))

Executing query: SELECT * FROM table1
Results for query: SELECT * FROM table1
Executing query: SELECT * FROM table2
Results for query: SELECT * FROM table2
CacheProxy: Returning cached result for query: SELECT * FROM table1
Results for query: SELECT * FROM table1
Executing query: SELECT * FROM table3
Results for query: SELECT * FROM table3


In [13]:
from abc import ABC, abstractmethod
from datetime import datetime
import time
import random

# Define an interface for the Real Subject
class Subject(ABC):
    @abstractmethod
    def perform_operation(self):
        pass

# Define the Real Subject that we want to monitor
class RealSubject(Subject):
    def perform_operation(self):
        """
        The Real Subject's method representing a costly operation.
        """
        print("RealSubject: Performing a costly operation...")
        time.sleep(random.random()*5)  # Sleep for a random (0-5) seconds

# Define the Monitoring Proxy that will lazily instantiate the Real Subject
class MonitoringProxy(Subject):
    def __init__(self):
        self._real_subject = None

    def perform_operation(self):
        """
        The Proxy's method, which monitors and adds performance metrics.
        Lazily instantiates the Real Subject on the first call.
        """
        if self._real_subject is None:
            print("MonitoringProxy: Lazy loading the RealSubject...")
            self._real_subject = RealSubject()

        start_time = datetime.now()
        print(f"MonitoringProxy: Recording operation start time - {start_time}")

        # Delegate the operation to the Real Subject
        self._real_subject.perform_operation()

        end_time = datetime.now()
        execution_time = end_time - start_time
        print(f"MonitoringProxy: Recording operation end time - {end_time}")
        print(f"MonitoringProxy: Operation executed in {execution_time} seconds")

# Client code
if __name__ == "__main__":
    # Create the Monitoring Proxy (lazy loading of Real Subject)
    monitoring_proxy = MonitoringProxy()

    # Client interacts with the Proxy
    monitoring_proxy.perform_operation()

MonitoringProxy: Lazy loading the RealSubject...
MonitoringProxy: Recording operation start time - 2024-05-20 13:23:18.716950
RealSubject: Performing a costly operation...
MonitoringProxy: Recording operation end time - 2024-05-20 13:23:19.401951
MonitoringProxy: Operation executed in 0:00:00.685001 seconds
